<a href="https://colab.research.google.com/github/Shakthi-2406/Yolo-nas/blob/main/yolonas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install super_gradients

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.1 MB/s eta 0:00:00
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4832 sha256=5706077019989958e685e4ac40a00b0027ea9d456fad4b903a9c95590ff657e0
  Stored in directory: /root/.cache/pip/wheels/a1/49/46/1b13a65d8da11238af9616b00fdde6d45b0f95d9291bac8452
  Created wheel for treelib: filename=treelib-1.6.1-py3-

In [3]:
!pip install torch

In [4]:
!pip install lap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lap: filename=lap-0.4.0-cp310-cp310-linux_x86_64.whl size=1655213 sha256=6f39030fa53b89e66b4daa146a661202d410652f2ea8dc7abc3d2d6b09f7f5a2
  Stored in directory: /root/.cache/pip/wheels/00/42/2e/9dfe19270eea279d79e84767ff0d7b8082c3bf776cad00e83d
Successfully built lap


In [5]:
!pip install tk

In [6]:
!pip install filterpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=42ac39143727343d636a228ce46960d18a8e11c5f1f62aac2d4db93ccfb88a33
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy


In [7]:
import cv2
from super_gradients.training import models
import torch
import numpy as np
import math
from tqdm.notebook import tqdm
from google.colab.patches import cv2_imshow
from sort import *

[2023-06-29 11:13:48] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-06-29 11:14:02] INFO - utils.py - NumExpr defaulting to 2 threads.
[2023-06-29 11:14:03] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-06-29 11:14:03] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-06-29 11:14:03] WARNING - export.py - Failed to import pytorch_quantization
[2023-06-29 11:14:03] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2023-06-29 11:14:03] WARNING - env_sanity_check.py - Failed to verify installed packages: boto3 required but not found
[2023-06-29 11:14:03] WARNING - env_sanity_check.py - Failed to verify installed packages: deprecated required but not found
[2023-06-29 11:14:03] WARNING - env_sanity_check.py - Failed to verify installed packages: coverage required but not found
[2023-06-29 11:14:03] WARNING

In [49]:
def draw_boxes(img, bbox, identities=None, names=None, offset=(0,0)):
  for i,box in enumerate(bbox):
    x1, y1, x2, y2 = [int(i) for i in box]
    x1 += offset[0]
    x2 += offset[0]
    y1 += offset[0]
    y2 += offset[0]
    id = int(identities[i]) if identities is not None else 0
    label = str(id) + ": Emp"
    (w,h) = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
    cv2.rectangle(img, (x1,y1), (x2,y2), (0, 255, 255), 2)
    cv2.rectangle(img, (x1,y1-20), (x1+w[0], y1), (255, 144, 30), -1)
    cv2.putText(img, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255,255,255], 1)
  return img

In [ ]:
# LINE_START = Point(140,0)
# LINE_END = Point(350,80)

In [9]:
def inside(px, py):
  # exp_y = 0.3658*px - 49.39
  exp_y = 0.3239*px - 1.619
  return py >= exp_y

In [26]:
cap = cv2.VideoCapture("4p-c1.avi")
frame_width, frame_height = int(cap.get(3)), int(cap.get(4))
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model = models.get("yolo_nas_l", pretrained_weights = "coco").to(device)
output = cv2.VideoWriter("YoloNasLWithCount.avi", cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width, frame_height))

frames_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(frames_count)

[2023-06-29 11:32:19] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.


3915


In [27]:
count = 0

# max age = no of frames we should wait for id x to come back once it goes out of frame
tracker = Sort(max_age = 20, min_hits = 3, iou_threshold = 0.3)
total_ins = []
total_outs = []
inCount = 0
outCount = 0

limit = [5, 0, 360, 115]

for _ in tqdm(range(frames_count)):
  ret, frame = cap.read()
  count += 1
  if ret:
    detections = np.empty((0,5))
    result = list(model.predict(frame, conf = 0.35, fuse_model = False))[0]
    bbox_xyxys = result.prediction.bboxes_xyxy.tolist()
    confidences = result.prediction.confidence
    labels = result.prediction.labels.tolist()
    for (bbox_xyxy, confidence, cls) in zip(bbox_xyxys, confidences, labels):
      classname = int(cls)
      bbox = np.array(bbox_xyxy)
      x1, y1, x2, y2 = bbox[0], bbox[1], bbox[2], bbox[3]
      x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
      conf = math.ceil((confidence*100))/100

      if classname == 0:
        currentArray = np.array([x1, y1, x2, y2, conf])
        detections = np.vstack((detections, currentArray))

    resultsTracker = tracker.update(detections)
    cv2.line(frame, (limit[0], limit[1]), (limit[2], limit[3]), (0, 255, 0), 3)
    for result in resultsTracker:
      x1, y1, x2, y2, id = result
      x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
      cx, cy = int((x1+x2)/2), y2
      cv2.circle(frame, (cx, cy), 5, (255, 0, 255), cv2.FILLED)
      cv2.rectangle(frame, (x1, y1), (x2, y2), (85, 45, 255), 1)
      label = f"{int(id)}"
      t_size = cv2.getTextSize(label, 0, fontScale = 1, thickness = 2)[0]
      c2 = x1 + t_size[0], y1 - t_size[1] - 3
      cv2.rectangle(frame, (x1, y1), c2, [255, 0, 255], -1, cv2.LINE_AA)
      cv2.putText(frame, label, (x1, y1-2), 0, 1, [255, 255, 255], thickness=1, lineType = cv2.LINE_AA)
      if total_ins.count(id) == 0:
        if inside(cx, cy):
          cv2.line(frame, (limit[0], limit[1]), (limit[2], limit[3]), (255, 0, 0), 3)
          inCount += 1
          total_ins.append(id)
          try:
            total_outs.remove(id)
          except ValueError:
            pass
      else:
        if not inside(cx, cy):
          cv2.line(frame, (limit[0], limit[1]), (limit[2], limit[3]), (255, 0, 0), 3)
          outCount += 1
          total_outs.append(id)
          try:
            total_ins.remove(id)
          except ValueError:
            pass
    cv2.putText(frame, f"In: {inCount} Out: {outCount}", (20, 35), cv2.FONT_HERSHEY_PLAIN, color=(0, 0, 0),fontScale = 1, thickness = 2)
    output.write(frame)
    # cv2_imshow(frame)
    # if inCount == 0:
      # break
    # if cv2.waitKey(1) & 0xFF == ord('1'):
      # break
  else:
    break

output.release()
cap.release()
cv2.destroyAllWindows()


  0%|          | 0/3915 [00:00<?, ?it/s]

KeyboardInterrupt: ignored